In [1]:
import numpy as np
from ppa.healthcare.system import get_system
import json
import pandas as pd

In [2]:
N_Test_SSM = 13914911
N_Test_Xpert = 4120552 
N_Det_Pub = 1688427
N_Det_Eng = 733694
N_DetBac = 513050 + 548981
N_DetCDx = 835930 + 1037210
N_Txi_Pub = 1527464
N_Txi_Eng = 554980
N_Pop = 1425775850

PropXpert_Eng = (215594 + 262160) / (3483130 + 2365739)
BacPerXpert_Eng = 68556 / (215594 + 262160)

N_Test_Xpert_Eng = N_Test_Xpert * PropXpert_Eng
N_DetBac_Eng = N_Test_Xpert_Eng * BacPerXpert_Eng

data = {
    'N_DetBac_Pub': N_DetBac - N_DetBac_Eng,
    'N_DetBac_Eng': N_DetBac_Eng,
    'N_Test_SSM_Pub': N_Test_SSM,
    'N_Test_Xpert_Pub': N_Test_Xpert - N_Test_Xpert_Eng,
    'N_Test_Xpert_Eng': N_Test_Xpert_Eng,
    'N_DetCDx_Pub': N_Det_Pub - (N_DetBac - N_DetBac_Eng),
    'N_DetCDx_Eng': N_Det_Eng - N_DetBac_Eng,
    'N_Txi_Pub': N_Txi_Pub,
    'N_Txi_Eng': N_Txi_Eng,
    'DrugTime_Pri': 0.0006037734 * N_Pop
}

data = {k: v / N_Pop for k, v in data.items()}
data

{'N_DetBac_Pub': 0.0007110044816354197,
 'N_DetBac_Eng': 3.38748765049219e-05,
 'N_Test_SSM_Pub': 0.009759536185158417,
 'N_Test_Xpert_Pub': 0.0026539742300999707,
 'N_Test_Xpert_Eng': 0.00023606770741776734,
 'N_DetCDx_Pub': 0.000473211817160776,
 'N_DetCDx_Eng': 0.0004807179327364466,
 'N_Txi_Pub': 0.001071321273957614,
 'N_Txi_Eng': 0.0003892477208110938,
 'DrugTime_Pri': 0.0006037734}

In [3]:
exo = {
    'sens_ssm': 0.64,
    'spec_ssm': 0.98,
    'sens_xpert': 0.85,
    'sens_xpert_ss-': 0.64,
    'spec_xpert': 0.98,
    'p_csi_pub': 0.483,
    'dur_pub': 0.5,
    'p_loss_sputum': 0.15,
    'p_txi_pub': 0.9,
    'p_txi_eng': 0.76,
    'spec_cdx': 0.8
}

In [4]:
with open('data/India/pars_cs_all.json', 'r') as f:
    pars = json.load(f)
    
pars, prev, txo = pars['pars'], pars['prev'], pars['txo']

In [5]:
ds = list()

for p0 in pars:
    p = dict(exo)
    p['sens_cdx'] = p0['sens_cdx']
    p['spec_cdx'] = p0['spec_cdx']

    p['p_ava_ssm_pub'] = p0['p_ava_ssm_pub']
    p['p_ava_xpert_pub'] = p0['p_ava_naat_pub']
    p['p_ava_xpert_eng'] = p0['p_ava_naat_eng']
    p['p_csi_ppm'] = p0['p_csi_ppm']
    p['p_csi_pub'] = p0['p_csi_pub']

    sys = get_system(p)

    d = dict()
    d['P1_Pub'], d['P2_Pub'], d['P3_Pub'], d['P4_Pub'] = sys.Public.Entry
    d['P3_Eng'], d['P4_Eng'] = sys.Engaged.Entry
    for i, alg in enumerate(sys.Public.Algorithms, 1):
        r0 = alg.dx(100, 0)
        bac = r0['N_Det_SSM'] + r0['N_Det_Xpert']
        cdx = r0.TruePos - bac
        pl = r0.FalseNegPreDx
        ng = r0.FalseNeg - pl


        d.update({
            f'P{i}_TP_Bac': bac,
            f'P{i}_TP_CDx': cdx,
            f'P{i}_FN': ng,
            f'P{i}_FN_PDxLTFU': pl
        })

        r1 = alg.dx(0, 100)
        bac = r1['N_Det_SSM'] + r1['N_Det_Xpert']
        cdx = r1.FalsePos - bac
        pl = r1.TrueNegPreDx
        ng = r1.TrueNeg - pl

        d.update({
            f'P{i}_FP_Bac': bac,
            f'P{i}_FP_CDx': cdx,
            f'P{i}_TN': ng,
            f'P{i}_TN_PDxLTFU': pl
        })

    ds.append(d)
    
ds = pd.DataFrame(ds)

In [6]:
ds.describe().T

,count,mean,std,min,25%,50%,75%,max
P1_Pub,2000.0,0.212922,4.876522e-02,0.077913,0.171163,0.217546,0.255951,0.289710
P2_Pub,2000.0,0.641324,5.079298e-02,0.448852,0.602677,0.652100,0.684656,0.708723
P3_Pub,2000.0,0.032814,1.866728e-02,0.000455,0.016153,0.032595,0.050023,0.070013
P4_Pub,2000.0,0.112940,8.098764e-02,0.001112,0.043232,0.097699,0.176229,0.403239
P3_Eng,2000.0,0.033192,5.623740e-03,0.016892,0.029145,0.033303,0.037586,0.044799
P4_Eng,2000.0,0.966808,5.623740e-03,0.955201,0.962414,0.966697,0.970855,0.983108
P1_TP_Bac,2000.0,73.984000,2.459093e-12,73.984000,73.984000,73.984000,73.984000,73.984000
P1_TP_CDx,2000.0,18.455020,4.222708e+00,6.180526,15.383227,18.637328,21.556421,26.015009
P1_FN,2000.0,3.201559,1.788029e+00,0.000419,1.888327,3.124364,4.502253,8.398969
P1_FN_PDxLTFU,2000.0,4.359421,2.434680e+00,0.000571,2.571252,4.254308,6.130519,11.436505


In [20]:
ds = list()

for p0 in pars:
    p = dict(exo)
    p['sens_cdx'] = p0['sens_cdx']
    p['spec_cdx'] = p0['spec_cdx']

    p['p_ava_ssm_pub'] = p0['p_ava_ssm_pub']
    p['p_ava_xpert_pub'] = p0['p_ava_naat_pub']
    p['p_ava_xpert_eng'] = p0['p_ava_naat_eng']
    p['p_csi_ppm'] = p0['p_csi_ppm']
    p['p_csi_pub'] = p0['p_csi_pub']
    

    sys = get_system(p)

    d = dict()
    d['p_entry_pub'], d['p_entry_eng'], d['p_entry_pri'] = sys.Entry
    d['sens_cdx'] = p0['sens_cdx']
    d['spec_cdx'] = p0['spec_cdx']
    d['p_txi_pub'] = p0['p_txi_pub']
    d['p_txi_eng'] = p0['p_txi_eng']
    d['p_txi_pri'] = p0['p_txi_pri']
    d['p_pri_on_pub'] = p0['p_pri_on_pub']
    
    
    for i, sector, ava in [('Pub', sys.Public, 1 - (1 - p['p_ava_ssm_pub']) * (1 - p['p_ava_xpert_pub'])), ('Eng', sys.Engaged, p0['p_ava_naat_eng']), ('Pri', sys.Private, 1)]:
        r0 = sector.seek_care(100, 0)

        d.update({
            f'{i}_TP': r0.TruePos,
            f'{i}_TP_Bac': (r0['N_Det_SSM'] + r0['N_Det_Xpert']) / ava,
            f'{i}_FN_Bac_Fail': r0['N_SampleFailed'] / ava,
        })

        r1 = sector.seek_care(0, 100)

        d.update({
            f'{i}_FP': r0.FalsePos,
            f'{i}_FP_Bac': (r1['N_Det_SSM'] + r1['N_Det_Xpert']) / ava,
            f'{i}_TN_Bac_Fail': r1['N_SampleFailed'] / ava,
        })
        
    r0 = sys.seek_care(100, 0)
    r0 = (r0['Public'] + r0['Engaged'] + r0['Private'])
    d['All_TP'] = r0.TruePos
    p_txi = np.array([d['p_txi_pub'], d['p_txi_eng'], d['p_txi_pri']])
    d['All_Txi'] = (sys.Entry * np.array([d['Pub_TP'], d['Eng_TP'], d['Pri_TP']]) * p_txi).sum()
    
    ds.append(d)
    
ds = pd.DataFrame(ds)
ds.describe().T

,count,mean,std,min,25%,50%,75%,max
p_entry_pub,2000.0,0.483000,2.293184e-14,0.483000,0.483000,0.483000,0.483000,0.483000
p_entry_eng,2000.0,0.304490,2.970146e-02,0.257691,0.282337,0.300583,0.323249,0.417508
p_entry_pri,2000.0,0.212510,2.970146e-02,0.099492,0.193751,0.216417,0.234663,0.259309
sens_cdx,2000.0,0.709372,1.623120e-01,0.237566,0.591299,0.716379,0.828583,0.999962
spec_cdx,2000.0,0.987735,7.130285e-03,0.966253,0.982716,0.987942,0.993319,0.999988
p_txi_pub,2000.0,0.904664,2.266540e-04,0.903872,0.904512,0.904664,0.904820,0.905409
p_txi_eng,2000.0,0.756463,4.968562e-04,0.754512,0.756112,0.756458,0.756818,0.757955
p_txi_pri,2000.0,0.667922,8.227105e-02,0.500217,0.604522,0.675807,0.735810,0.799775
p_pri_on_pub,2000.0,0.329379,1.931113e-01,0.002243,0.170567,0.300462,0.473087,0.868655
Pub_TP,2000.0,86.095078,8.207815e+00,61.129162,80.623410,86.618457,92.273582,99.998417


In [8]:
r0 = sys.Public.seek_care(100, 0)
r0.print(True)

-- TP/FN (FN pre-dx) = 78/22 (8)
-- FP/TN (TN pre-dx)= 0/0 (0)
-- PPV = 100.00%
-- Negative due to pre-dx LTFU = 34.10%
-- N_collect_Sputum: 96.19876585071991
-- N_test_SSM: 80.5480655
-- N_Det_SSM: 51.55076192
-- N_test_Xpert_ss-: 7.952135571679711
-- N_Det_Xpert: 6.127119418020143
-- N_SampleFailed: 14.429814877607985
-- N_test_CDx: 42.32211866197985
-- N_PreDxLTFU: 7.663570642532333
-- N_Eval_TB_SSM > Xpert > CDx: 25.987371149280104
-- N_Eval_NonTB_SSM > Xpert > CDx: 0.0
-- N_Det_TB_SSM > Xpert > CDx: 22.39671945981482
-- N_Det_NonTB_SSM > Xpert > CDx: 0.0
-- N_Eval_TB_SSM > CDx: 68.7750588507199
-- N_Eval_NonTB_SSM > CDx: 0.0
-- N_Det_TB_SSM > CDx: 52.11923117404588
-- N_Det_NonTB_SSM > CDx: 0.0
-- N_test_Xpert: 1.2208854731119156
-- N_Eval_TB_Xpert > CDx: 1.4363358507199009
-- N_Eval_NonTB_Xpert > CDx: 0.0
-- N_Det_TB_Xpert > CDx: 1.224651187741701
-- N_Det_NonTB_Xpert > CDx: 0.0
-- N_Eval_TB_CDx: 3.801234149280101
-- N_Eval_NonTB_CDx: 0.0
-- N_Det_TB_CDx: 1.7824260994956558
-- N_

-- TP/FN (FN pre-dx) = 62/38 (4)
-- FP/TN (TN pre-dx)= 0/0 (0)
-- PPV = 100.00%
-- Negative due to pre-dx LTFU = 10.00%
